    实现功能：文档解析
    输入：Microsoft Word、PDF
    输出：文档目录、文档内图片（PNG格式）、文档表格（采用pandas的dataFrame数据类型表示）、可返回的文档版面数据

In [1]:
import pdfplumber
import pdfminer.high_level
import fitz
import docx
import pandas as pd
import os, re

    功能：从pdf中提取表格
    输入：pdf文件路径pdf_path（"example.pdf"）
    输出：对于每一个表输出一个pandas的dataFrame类型，采用print方式
    返回值：无

In [2]:
def pdf_extract_tables(pdf_path):
    
    #open pdf with pdfplumber
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            for table in page.extract_tables():
                
                #for every table extract data to dataFrame
                df = pd.DataFrame(table[1:], columns=table[0])
                print(df)
        pdf.close()

#for debugging
#pdf_extract_tables("demo.pdf")

    功能：从pdf中提取图片（从pdf文件中提取所有图片，并将它们保存为png文件）
    输入：pdf文件路径pdf_path（"example.pdf"），图片保存地址result_path
    输出：对于每一个图片以png格式直接输出到result_path
    返回值：无

In [3]:
def pdf_extract_images(pdf_path, result_path):
    
    #open pdf with fitz
    doc = fitz.open(pdf_path)
    
    #make sure the result path exists
    if not os.path.exists(result_path):
        os.makedirs(result_path)
        
    #extract the images
    for page in doc:
        image_list = page.get_images()
        for image_index, img in enumerate(image_list):
            xref = img[0]
            pix = fitz.Pixmap(doc, xref)
            
            #if the colors obey the RGB format
            if pix.n < 5:
                pix.save(result_path + "/p%s-%s.png" % (page.number, xref))
                
            #else transform to RGB format
            else:
                pix1 = fitz.Pixmap(fitz.csRGB, pix)
                pix1.save(result_path + "/p%s-%s.png" % (page.number, xref))
                pix1 = None
            pix = None

#for debugging
#pdf_extract_images("demo.pdf", "demo_pdf_images")

    功能：从pdf中提取目录并输出
    输入：pdf文件路径pdf_path（"example.pdf"）
    输出：一个list对象，包括每个章节的标题
    返回值：无

In [59]:
from pdfminer.high_level import extract_text
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument

def pdf_get_toc(pdf_path):

    #open pdf file and new a PDFParser object
    fp = open(pdf_path, 'rb')
    parser = PDFParser(fp)
    document = PDFDocument(parser)
    
    #get outlines
    outlines = document.get_outlines()
    result = []
    
    #append title in result list
    for (level,title,dest,a,se) in outlines:
        result.append(title)
    print(result)

#for debugging
#pdf_get_toc("demo.pdf")

    功能：从pdf中提取正文并输出
    输入：pdf文件路径pdf_path（"example.pdf"）
    输出：输出正文内容
    返回值：无

In [5]:
def get_pdf_text(pdf_path):
    
    #extract the plaintext
    text = pdfminer.high_level.extract_text(pdf_path)
    print(text)

#for debugging
#get_pdf_text("demo.pdf")

    功能：从Microsoft Word中提取表格
    输入：Microsoft Word文件路径doc_path（"example.docx"）
    输出：对于每一个表输出一个pandas的dataFrame类型，采用print方式
    返回值：无

In [11]:
def docx_extract_tables(doc_path):
    
    #open the document and new a document object to handle it
    document = docx.Document(doc_path)
    result = []
    
    #transform the table to a list
    for tb in document.tables:
        for i, row in enumerate(tb.rows):
            temp = []
            for j, cell in enumerate(row.cells):
                text = ''
                for p in cell.paragraphs:
                    text += p.text
                temp.append(text)
            result.append(temp)
        
        #use a dataFrame object to handle the result
        df = pd.DataFrame(result[1:], columns=result[0])
        print(df)

#for debugging
#docx_extract_tables("demo.docx")

    功能：从Microsoft Word中提取图片（从docx文件中提取所有图片，并将它们保存为PNG文件）
    输入：Microsoft Word文件路径doc_path（"example.docx"）
    输出：对于每一个图片以png格式直接输出到指定路径result_path
    返回值：无

In [12]:
def docx_extract_images(word_path, result_path):
    
    #create a document object to handle the doc file
    doc = docx.Document(word_path)
    
    #extract the rels
    dict_rel = doc.part._rels
    for rel in dict_rel:
        rel = dict_rel[rel]
        if "image" in rel.target_ref:
            
            #make sure the result path exists
            if not os.path.exists(result_path):
                os.makedirs(result_path)
                
            #spawn the image name
            img_name = re.findall("/(.*)", rel.target_ref)[0]
            word_name = os.path.splitext(word_path)[0]
            if os.sep in word_name:
                new_name = word_name.split('\\')[-1]
            else:
                new_name = word_name.split('/')[-1]
            img_name = f'{new_name}-'+f'{img_name}'
            
            #create the image file and write in the content
            with open(f'{result_path}/{img_name}', "wb") as f:
                f.write(rel.target_part.blob)

#for debugging
#docx_extract_images("demo.docx", "demo_docx_images")

    功能：从Microsoft Word中提取目录并输出
    输入：Microsoft Word文件路径doc_path（"example.docx"）
    输出：一个list对象，包括每个章节的标题与等级
    返回值：无

In [51]:
from docx import Document
from docx.shared import Cm

def docx_extract_toc(doc_path):
    
    #create a document object to handle the doc
    document = Document(doc_path)
    result = []
    
    #extract the toc with the label 'Heading'
    for paragraph in document.paragraphs:    
        if paragraph.style.name.startswith('Heading'):
            print(paragraph.style.name.split(' ')[1] + ' ' + paragraph.text + '……')
            
#for debugging
docx_extract_toc("demo.docx")

1 我不知道是什么……
2 我也不知道是什么……
2 我管他是什么……
1 这是什么……
1 这是一张图……
1 这是一个表……
1 没了……


    功能：从Microsoft Word中提取正文并输出
    输入：Microsoft Word文件路径doc_path（"example.docx"）
    输出：输出正文内容
    返回值：无

In [6]:
from docx import Document

def docx_extract_text(doc_path):
    
    #extract the plaintext
    document = Document(doc_path)
    result = []
    for paragraph in document.paragraphs:
        result.append(paragraph.text)
    print(result)

#for debugging
#docx_extract_text("demo.docx")